# Novartis

## Beachten im nächsten Jahr
* OCR

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# Read pdf into DataFrame
#columns = [48.53, 106.31, 163.7, 221.22, 279.07, 337.85, 395.94, 454.52, 511.85, 569.83, 628.18, 744.27]
columns = [46.24, 106.31, 163.7, 221.22, 279.07, 337.85, 395.94, 454.52, 511.85, 569.83, 628.18, 744.27]
# "x1": 46.69,
area = {
  "x1": 22.298105130004696,
  "x2": 808.6649800537107,
  "y1": 193.18539842481607,
  "y2": 566.8938865077018,
}

df = tabula.read_pdf("Novartis_payment-disclosure-pharma-switzerland-2017-de_ocr.pdf", pages='all', stream=True, columns=columns, guess=False, area=[area['y1'], area['x1'], area['y2'], area['x2']])

## Format Table

In [35]:
#Rename Columns
df_export = df.copy()

#Rename
df_export.columns = ['empty', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total']

########################
#
#  ROUND 1
#  Concat Row above and beneath a row with total value
#
########################

df_export['name'] = df_export['name'].fillna("")
df_export['address'] = df_export['address'].fillna("")


write_to_excel(df_export, 'tmp.xlsx')

selection = df_export[df_export.total.notna()]
remove_index = []

for index, row in selection.iterrows():
    
    r_previous = df_export.loc[index - 1]
    r_next = df_export.loc[index + 1]
    
    #Concat Name
    if r_previous['name'] != "":
        row['name'] = r_previous['name'] + ' ' + row['name']
        row['name'] = row['name'].strip()
        
    if r_next['name'] != "":
        row['name'] = row['name'] + ' ' + r_next['name']
        row['name'] = row['name'].strip()
    
        
    #Concat address
    if not pd.isnull(r_previous['address']):
        row['address'] = r_previous['address'] + ' ' + row['address']
        row['address'] = row['address'].strip()
        
    if not pd.isnull(r_next['address']):
        row['address'] = row['address'] + ' ' + r_next['address']
        row['address'] = row['address'].strip()

    df_export.loc[index, 'name'] = row['name']
    df_export.loc[index, 'address'] = row['address']
    
    remove_index.append(index - 1)
    remove_index.append(index + 1)

#Remove Rows
df_export = df_export.drop(df_export.index[remove_index])

#Replace empty to NaN
df_export.loc[df_export['name'] == '', 'name'] = np.NaN
df_export.loc[df_export['address'] == '', 'address'] = np.NaN
df_export.loc[df_export['donations_grants'] == '----', 'donations_grants'] = np.NaN
df_export.loc[df_export['donations_grants'] == '.', 'donations_grants'] = np.NaN
df_export.loc[df_export['donations_grants'] == '·', 'donations_grants'] = np.NaN

#Remove empty rows
df_export.drop(columns=['empty'], inplace=True)
df_export = df_export.dropna(how='all')

#Reset Index
df_export = df_export.reset_index(drop=True)

########################
#
#  ROUND 2
#  Concat Rows without name. Logic: If Number at the end, concat it with row above (its probably an address)
#
########################

def concat_rows_up_or_down(df_export):
    #Fill na
    df_export['name'] = df_export['name'].fillna("")
    df_export['address'] = df_export['address'].fillna("")

    selection = df_export[df_export['name'] == ""]
    selection = selection.sort_index(axis=0 ,ascending=False)

    for index, row in selection.iterrows():
        address = df_export.loc[index, 'address']
        if re.search(r'\d+$', str(address)):
            #Add to row above
            df_export.loc[index - 1, 'address'] = df_export.loc[index - 1, 'address'] + ' ' + address
        else:
            #Copy to above
            df_export.loc[index + 1, 'address'] = address + " " + df_export.loc[index + 1, 'address']

        #df_export.loc[index, 'address'] = "REPLACED"
        df_export.loc[index, 'address'] = ""

    #Remove empty rows
    df_export.loc[df_export['address'] == '', 'address'] = np.NaN
    df_export.loc[df_export['name'] == '', 'name'] = np.NaN
    df_export = df_export.dropna(how='all')
    
    return df_export

#df_export = concat_rows_up_or_down(df_export)
#df_export = concat_rows_up_or_down(df_export)
#df_export = concat_rows_up_or_down(df_export)

#df_export[7:10]

In [27]:
s = ''
r = re.search(r'\d+$', 'Hauptstr8sse d')
if r:
    print("yes")

In [69]:
#Rename Columns
df_export = df.copy()
df_export.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'total', 'empty1', 'empty2']

#Shift all where 'name' is empty
df_export[df_export['name'].isna()] = df_export[df_export['name'].isna()].shift(-1, axis='columns')

#Type by text
index_hco = df_export[df_export['name'].str.contains("INDIVIDUAL NAMED DISCLOSURE", na=False)].index[1]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove empty rows
df_export = df_export.dropna(subset=['location'], how='all')
df_export = df_export[~df_export.name.str.contains('Full Name')]
df_export = df_export[0: -3]

#Add Rows Manually
df_export = add_rows_manually(df_export)

#Remove empties
df_export.drop(columns=['empty1', 'empty2'], inplace=True)

#Cleanup Numbers and sum
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Add empty fields
df_export = add_plz(df_export)

#Remove Carination
df_export = remove_carination(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'mundipharma')

saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv', True)
#write_to_excel(df_export, 'tmp.xlsx')